In [ ]:
import sqlite3
import pandas as pd812
from cryptography.fernet import Fernet
import pandas as pd

DB_FILE = "telematics.db"
KEY_FILE = "secret.key"

conn = sqlite3.connect(DB_FILE)

df = pd.read_sql("SELECT * FROM drivers", conn)

features = pd.read_sql("SELECT * FROM trips", conn)

telemetry = pd.read_sql("SELECT * FROM telemetry_secure", conn)

conn.close()

print("✅ Loaded from DB")
print("Drivers:", df.shape)
print("Trips:", features.shape)
print("Telemetry:", telemetry.shape)

def load_key(key_file=KEY_FILE):
    with open(key_file, "rb") as f:
        return f.read()

def decrypt_value(value, fernet):
    if value is None:
        return None
    return float(fernet.decrypt(value.encode()).decode())

key = load_key()
fernet = Fernet(key)

telemetry["lat_dec"] = telemetry["lat"].apply(lambda x: decrypt_value(x, fernet))
telemetry["lon_dec"] = telemetry["lon"].apply(lambda x: decrypt_value(x, fernet))

display(telemetry)


✅ Loaded from DB
Drivers: (30, 23)
Trips: (300, 12)
Telemetry: (209104, 11)


,id,timestamp,trip_id,driver_id,lat,lon,speed,acceleration,road_type,engine_on,geohash,lat_dec,lon_dec
0,1,2025-09-07 01:17:00,driver_1_trip_1,driver_1,gAAAAABowmWylTbbXDlUHl-zlmxBNt-jwaK-tanrmV6slS...,gAAAAABowmW1Yr8susleq1X8ZsWJiK1U_Q-2e_VshY3dv9...,9.905877,-2.343964,residential,1,9muz8,33.678795,-117.062931
1,2,2025-09-07 01:17:05,driver_1_trip_1,driver_1,gAAAAABowmWyT5Ax185kRaethjl1hllfBMcHPoRj3vBWve...,gAAAAABowmW12Wy3gScW8eSQq5_8b1NZIsJLGy-JQygtku...,11.761901,0.371205,residential,1,9ub15,26.901010,-101.083936
2,3,2025-09-07 01:17:10,driver_1_trip_1,driver_1,gAAAAABowmWy-3FjlbDklgqPgBw9ukct7WmBWsLJZhKxa0...,gAAAAABowmW1YbH3HF1EaHZOmg8i414WiefDODSCmPCIaV...,12.201238,0.087868,residential,1,dq7rh,36.428386,-74.003414
3,4,2025-09-07 01:17:15,driver_1_trip_1,driver_1,gAAAAABowmWyl2Cg91Sa2nFSaUn5nDSZ9cHZ5KXKwRzlsV...,gAAAAABowmW1bWa1sIFAuWVfjpOkx_uXdZxxdZOsnS8Vyg...,20.073079,1.574368,residential,1,9mu5p,32.871377,-117.781032
4,5,2025-09-07 01:17:20,driver_1_trip_1,driver_1,gAAAAABowmWyrsNscrmYnHEENxY4VJ9ZbbL9tujzXW0ep8...,gAAAAABowmW18Ob4NkSDACi9udXQrZVTsoMyAj5Y0frJLh...,8.429803,-2.328655,residential,1,dr7v2,41.743242,-73.433907
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209099,209100,2025-09-05 05:19:05,driver_30_trip_10,driver_30,gAAAAABowmYJXRybYpeS4ggCAw46VZmsCcANcJJ6C5TBBd...,gAAAAABowmYLtqlnXoaSp4g7Wxs4yVPtdfJ8mJHoVEYdsI...,49.637773,-3.356191,highway,1,c2e34,48.015401,-119.085536
209100,209101,2025-09-05 05:19:10,driver_30_trip_10,driver_30,gAAAAABowmYJbdE1Nf0VaI2ibziWNzZqgFJTPR0j5V-mMI...,gAAAAABowmYLp5Q5fpoo7Y0PmLL3ZqEojkutdC5NojKqDa...,60.835609,2.239567,highway,1,f2wcr,48.057874,-68.942512
209101,209102,2025-09-05 05:19:15,driver_30_trip_10,driver_30,gAAAAABowmYJxuUeVBiH9u9GWLrlwgPVLNouuWq7NuHorc...,gAAAAABowmYLSmJiWyzwd4xzW4OxnuyX71UVk3ijOm4d4r...,53.181163,-1.530889,highway,1,dnky8,36.303880,-83.298447
209102,209103,2025-09-05 05:19:20,driver_30_trip_10,driver_30,gAAAAABowmYJ2G9TQc4XyPQjh5NESRTjMlTCo8srUI93PX...,gAAAAABowmYL2Y7YZT22RM0W5Qe-gdk4OTFGAr2wzZDlob...,53.840610,0.131889,highway,1,f0jm1,45.888556,-82.568962


In [ ]:
import numpy as np

def enhanced_risk_score(df):
    weights = {
        'total_harsh_brakes': 20,
        'total_harsh_accels': 15,
        'max_speed_overall': 20,
        'night_trip_pct_overall': 10,
        'claims_weighted_score': 25,
    }
    
    max_vehicle_risk = 15
    vehicle_risk_map = {'Sedan': 0, 'SUV': 5, 'Sports Car': 15, 'Truck': 10, 'Electric': -5}

    norm_features = {}
    for feature in weights.keys():
        min_val = df[feature].min()
        max_val = df[feature].max()
        range_val = max_val - min_val if max_val > min_val else 1
        norm_features[feature] = (df[feature] - min_val) / range_val

    vehicle_risk_raw = df['vehicle_type'].map(vehicle_risk_map).fillna(0)
    vehicle_risk_norm = (vehicle_risk_raw + 5) / (max_vehicle_risk + 5)  

    weighted_sum = np.zeros(len(df))
    for feature, weight in weights.items():
        weighted_sum += norm_features[feature] * weight

    total_weight = sum(weights.values())
    vehicle_weight = 100 - total_weight
    weighted_sum += vehicle_risk_norm * vehicle_weight

    score = weighted_sum.clip(0, 100)

    return score


In [3]:
df["enhanced_risk_score"] = enhanced_risk_score(df)

In [ ]:
print(df.columns.tolist())


['driver_id', 'num_trips', 'total_miles', 'total_drive_time_min', 'avg_trip_duration_min', 'avg_trip_miles', 'avg_speed_overall', 'max_speed_overall', 'total_harsh_brakes', 'total_harsh_accels', 'avg_num_harsh_brakes', 'avg_num_harsh_accels', 'night_trip_pct_overall', 'idling_pct_overall', 'urban_pct_overall', 'highway_pct_overall', 'years_driving', 'num_claims', 'num_violations', 'vehicle_age', 'vehicle_type', 'insurance_policy_length_years', 'claims_weighted_score', 'enhanced_risk_score']


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

X = df.drop(columns=['driver_id', 'enhanced_risk_score'])
y = df['enhanced_risk_score']


categorical_features = ['vehicle_type']
numeric_features = [col for col in X.columns if col not in categorical_features]

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', 'passthrough', numeric_features)
    ]
)

catboost_model = CatBoostRegressor(
    random_seed=42,
    cat_features=[X.columns.get_loc(c) for c in categorical_features],
    verbose=0
)

rf_model = RandomForestRegressor(random_state=42)
xgb_model = XGBRegressor(random_state=42, verbosity=0)
gbr_model = GradientBoostingRegressor(random_state=42)

estimators = [('rf', rf_model), ('xgb', xgb_model), ('gbr', gbr_model)]
stack_model = StackingRegressor(
    estimators=estimators,
    final_estimator=Ridge(alpha=1.0),
    passthrough=True,
    n_jobs=-1
)

rf_pipeline = Pipeline([('preprocessor', preprocessor), ('model', rf_model)])
xgb_pipeline = Pipeline([('preprocessor', preprocessor), ('model', xgb_model)])
gbr_pipeline = Pipeline([('preprocessor', preprocessor), ('model', gbr_model)])
stack_pipeline = Pipeline([('preprocessor', preprocessor), ('model', stack_model)])

models = {
    "CatBoost": catboost_model,
    "RandomForest": rf_pipeline,
    "XGBoost": xgb_pipeline,
    "GradientBoosting": gbr_pipeline,
    "StackingEnsemble": stack_pipeline
}

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scoring = {
    "MAE": make_scorer(mean_absolute_error, greater_is_better=False),
    "RMSE": make_scorer(lambda yt, yp: np.sqrt(mean_squared_error(yt, yp)), greater_is_better=False),
    "R2": make_scorer(r2_score)
}
kf = KFold(n_splits=5, shuffle=True, random_state=42)

def cross_validate_model(model, X, y):
    mae_scores = cross_val_score(model, X, y, cv=kf, scoring=scoring["MAE"], n_jobs=-1)
    rmse_scores = cross_val_score(model, X, y, cv=kf, scoring=scoring["RMSE"], n_jobs=-1)
    r2_scores = cross_val_score(model, X, y, cv=kf, scoring=scoring["R2"], n_jobs=-1)
    return {
        "MAE": -np.mean(mae_scores),
        "RMSE": -np.mean(rmse_scores),
        "R²": np.mean(r2_scores)
    }

print("=== Cross-Validation Results (5-Fold Average) ===")
cv_results = {}
for name, model in models.items():
    print(f"Running CV for {name}...")
    cv_results[name] = cross_validate_model(model, X, y)

cv_df = pd.DataFrame(cv_results).T
print(cv_df)

print("\n=== Hold-Out Test Results ===")
for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    print(f"{name} -> MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}")

def calculate_realistic_premium(risk_score, base_premium=2285, max_risk_score=100):
    """
    Calculate a realistic insurance premium based on predicted risk score.
    Base premium is $2,285/year (average full coverage).
    """
    normalized_risk = np.clip(risk_score / max_risk_score, 0, 1)
    scaling_factor = 1 + 0.5 * normalized_risk  
    premium = base_premium * scaling_factor
    return premium

best_model = stack_pipeline
predicted_risk = best_model.predict(X_test)

premiums = [calculate_realistic_premium(r) for r in predicted_risk]

output_df = X_test.copy()
output_df['predicted_risk_score'] = predicted_risk
output_df['premium_annual'] = premiums
output_df['premium_monthly'] = output_df['premium_annual'] / 12

print("\n=== Sample Premium Output ===")
print(output_df[['predicted_risk_score', 'premium_annual', 'premium_monthly']].head())


=== Cross-Validation Results (5-Fold Average) ===
Running CV for CatBoost...
Running CV for RandomForest...
Running CV for XGBoost...
Running CV for GradientBoosting...
Running CV for StackingEnsemble...
                       MAE       RMSE        R²
CatBoost          8.488642   9.852129 -0.021176
RandomForest      8.839239  10.292795 -0.152049
XGBoost           9.269547  10.936334 -0.384808
GradientBoosting  9.895270  11.180327 -0.489254
StackingEnsemble  2.676506   3.157082  0.801570

=== Hold-Out Test Results ===
Training CatBoost...
CatBoost -> MAE: 5.9872, RMSE: 6.5334, R²: 0.6254
Training RandomForest...
RandomForest -> MAE: 6.2625, RMSE: 7.1456, R²: 0.5519
Training XGBoost...
XGBoost -> MAE: 9.0612, RMSE: 9.9547, R²: 0.1303
Training GradientBoosting...
GradientBoosting -> MAE: 9.1663, RMSE: 9.7184, R²: 0.1711
Training StackingEnsemble...
StackingEnsemble -> MAE: 1.8475, RMSE: 2.2762, R²: 0.9545

=== Sample Premium Output ===
    predicted_risk_score  premium_annual  premium_mon

In [ ]:
import joblib

joblib.dump(stack_pipeline, "stacking_model.pkl")


['stacking_model.pkl']